<a href="https://colab.research.google.com/github/amanupg/Imageclef-2024/blob/main/MobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3, VGG19, ResNet50, Xception, EfficientNetB0, MobileNetV2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import pandas as pd
import pathlib
from sklearn.cluster import KMeans
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False)

9406464/9406464 [==============================] - 1s 0us/step


In [ ]:
input_shape = (256, 256, 1)  # Grayscale images
input_layer = tf.keras.layers.Input(shape=input_shape)
gray_convert = tf.keras.layers.Lambda(lambda x: tf.image.grayscale_to_rgb(x))(input_layer)

# Pass through InceptionV3 base model
x = base_model(gray_convert)

# Add custom classification head
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

In [ ]:
model = Model(inputs=input_layer, outputs=predictions)

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    directory='/content/drive/MyDrive/Machine_Learning/Research/Train',
    labels='inferred',
    label_mode='int',
    batch_size=16,
    image_size=(256, 256),
    color_mode='grayscale'
)
validation_ds = keras.utils.image_dataset_from_directory(
    directory='/content/drive/MyDrive/Machine_Learning/Research/Test',
    labels='inferred',
    label_mode='int',
    batch_size=16,
    image_size=(256, 256),
    color_mode='grayscale'
)

Found 540 files belonging to 3 classes.
Found 60 files belonging to 3 classes.


In [ ]:
def process(image, label):
    image = tf.cast(image / 255, tf.float32)
    label = tf.one_hot(label, depth=3)  # One-hot encode labels for 3 classes
    return image, label


train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [ ]:
model_path = "/content/drive/MyDrive/Machine_Learning/Research/Model/mobile.h5"
csv_path = "/content/drive/MyDrive/Machine_Learning/Research/Model/mobile.log"
callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=False),
    ]

model.fit(
    train_ds,
    epochs=200,
    validation_data=validation_ds,
    callbacks=callbacks,
    shuffle=False
)

Epoch 1/200
34/34 [==============================] - ETA: 0s - loss: 0.2780 - accuracy: 0.9333
Epoch 1: val_loss improved from inf to 38.43827, saving model to /content/drive/MyDrive/Machine_Learning/Research/Model/mobile.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


34/34 [==============================] - 57s 551ms/step - loss: 0.2780 - accuracy: 0.9333 - val_loss: 38.4383 - val_accuracy: 0.3333
Epoch 2/200
34/34 [==============================] - ETA: 0s - loss: 0.1262 - accuracy: 0.9778
Epoch 2: val_loss improved from 38.43827 to 2.02286, saving model to /content/drive/MyDrive/Machine_Learning/Research/Model/mobile.h5
34/34 [==============================] - 3s 83ms/step - loss: 0.1262 - accuracy: 0.9778 - val_loss: 2.0229 - val_accuracy: 0.6833
Epoch 3/200
34/34 [==============================] - ETA: 0s - loss: 0.1249 - accuracy: 0.9759
Epoch 3: val_loss did not improve from 2.02286
34/34 [==============================] - 2s 55ms/step - loss: 0.1249 - accuracy: 0.9759 - val_loss: 12.4773 - val_accuracy: 0.3667
Epoch 4/200
34/34 [==============================] - ETA: 0s - loss: 0.0089 - accuracy: 1.0000
Epoch 4: val_loss did not improve from 2.02286
34/34 [==============================] - 2s 55ms/step - loss: 0.0089 - accuracy: 1.0000 - val

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Machine_Learning/Research/Model/mobile.h5')

In [ ]:
# Remove the last layer of the model
feature_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)

# Extract features from the test dataset
test_features = feature_extractor.predict(validation_ds)

num_clusters = 4  # specify the number of clusters
kmeans = KMeans(n_clusters=num_clusters)
clusters = kmeans.fit_predict(test_features)

# clusters variable now contains the cluster labels for each image in the test dataset

kmeans.labels_

4/4 [==============================] - 1s 11ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([1, 0, 2, 0, 2, 1, 1, 1, 0, 1, 1, 0, 1, 2, 0, 0, 1, 0, 1, 2, 0, 2,
       1, 1, 0, 0, 0, 2, 0, 2, 0, 0, 1, 3, 0, 2, 0, 3, 2, 1, 1, 0, 2, 1,
       2, 2, 2, 1, 0, 2, 0, 1, 2, 1, 0, 3, 1, 1, 2, 3], dtype=int32)

In [ ]:
data_dir = '/content/drive/MyDrive/Machine_Learning/Research/ImageCLEFmedical  GANs 2024 - Task2 test/generated_images'
data_dir = pathlib.Path(data_dir)

In [ ]:
image_count = len(list(data_dir.glob('*.*')))

# Create a dataset of image file paths
list_ds = tf.data.Dataset.list_files(str(data_dir/'*'), shuffle=False)
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)

In [ ]:
def process_path(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=1)  # Use decode_png if images are png format
    img = tf.image.resize(img, [256, 256])
    img = tf.expand_dims(img, axis=-1)  # Add a channel dimension
    img = (img / 255.0)  # Normalize pixels to 0,1
    return img


In [ ]:
new_dataset = list_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)


In [ ]:
# Remove the last layer of the model
feature_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)

# Batch the dataset
new_dataset = new_dataset.batch(16)  # Replace 16 with your desired batch size

# Extract features from the new dataset
new_features = feature_extractor.predict(new_dataset)

# Perform clustering on the extracted features
num_clusters = 4  # specify the number of clusters
kmeans = KMeans(n_clusters=num_clusters)
new_clusters = kmeans.fit_predict(new_features)

# new_clusters variable now contains the cluster labels for each image in the new dataset
print(kmeans.labels_)


190/190 [==============================] - 29s 148ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[1 1 0 ... 3 1 1]


In [ ]:
print(kmeans.labels_[:20])

[1 1 0 0 0 3 0 1 1 3 2 1 1 2 1 0 2 3 3 1]


In [ ]:
# Load the provided run.csv file
provided_run_path = '/content/drive/MyDrive/Machine_Learning/Research/ImageCLEFmedical  GANs 2024 - Task2 test/mobilenet.csv'

# Read the file without header
provided_run_df = pd.read_csv(provided_run_path, header=None, names=['data'])

# Map figure IDs to cluster labels
figure_ids = provided_run_df['data'].str.split('\t', expand=True)[0]  # Extract figure IDs
cluster_mapping = dict(zip(figure_ids, new_clusters + 1))  # Adding 1 to clusters to match the required labels [1, 2, 3, 4]

# Create a DataFrame with image IDs and corresponding cluster mappings
result_df = pd.DataFrame({'ImageID': figure_ids, 'Cluster': [cluster_mapping[x] for x in figure_ids]})

# Save the DataFrame to the same CSV file (overwrite the existing file)
result_df.to_csv(provided_run_path, index=False, header=False)
